In [103]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [6]:
import pandas as pd
import yaml

In [31]:
with open("cp.alerts.1", "r") as f:
    alerts = yaml.load(f.read(), yaml.SafeLoader)

In [40]:
print(yaml.dump(alerts[:10]))

- alert_info:
    communication_alert_type: EXTERNAL_COMMUNICATION_ATTEMPTED
    process_alert_type: EXISTING_PROCESS_COMPROMISED
    re_open_count: 1
    status: OPEN
  client:
    araali:
      app: default.canveyor.canveyor
      binary_name: /usr/local/bin/node
      namespace: default
      parent_process: containerd-shim
      process: node
      unmapped_app: default.canveyor.canveyor
      zone: cloud1_viriciti_com
  direction: NON_ARAALI_EGRESS
  ports:
    active_ports:
    - 443
  server:
    non_araali:
      organization: AMAZON-02
      server:
        dst_port: 443
        subnet:
          net_mask: 32
          subnet: 52.218.112.152
  timestamp: '2022-11-16T03:25:41Z'
  unique_id: id://cloud1_viriciti_com,:default.canveyor.canveyor:,node,containerd-shim,/usr/local/bin/node+++AMAZON-02:443+++false+++false
- alert_info:
    communication_alert_type: EXTERNAL_COMMUNICATION_ATTEMPTED
    process_alert_type: NEW_PROCESS_SEEN
    re_open_count: 1
    status: OPEN
  client:


In [33]:
# first seen?
# last seen > few days and IP in server -> snooze

In [132]:
df = pd.DataFrame(alerts_to_obj(alerts))

In [133]:
df

,alert_info.communication_alert_type,alert_info.process_alert_type,alert_info.re_open_count,alert_info.status,client.araali.app,client.araali.binary_name,client.araali.namespace,client.araali.parent_process,client.araali.process,client.araali.unmapped_app,client.araali.zone,direction,ports.active_ports,server.non_araali.organization,server.non_araali.server.dst_port,server.non_araali.server.subnet.net_mask,server.non_araali.server.subnet.subnet,timestamp,unique_id,server.non_araali.server.dns_pattern
0,EXTERNAL_COMMUNICATION_ATTEMPTED,EXISTING_PROCESS_COMPROMISED,1,OPEN,default.canveyor.canveyor,/usr/local/bin/node,default,containerd-shim,node,default.canveyor.canveyor,cloud1_viriciti_com,NON_ARAALI_EGRESS,[443],AMAZON-02,443,32,52.218.112.152,2022-11-16T03:25:41Z,"id://cloud1_viriciti_com,:default.canveyor.can...",NaN
1,EXTERNAL_COMMUNICATION_ATTEMPTED,NEW_PROCESS_SEEN,1,OPEN,default.thanos-store-gateway.thanos,/bin/thanos,default,containerd-shim,thanos,default.thanos-store-gateway.thanos,cloud1_viriciti_com,NON_ARAALI_EGRESS,[443],AMAZON-02,443,32,52.218.100.160,2022-11-16T03:25:45Z,"id://cloud1_viriciti_com,:default.thanos-store...",NaN
2,EXTERNAL_COMMUNICATION_ATTEMPTED,EXISTING_PROCESS_COMPROMISED,1,OPEN,default.time-api.time-api,/usr/local/bin/node,default,containerd-shim,node,default.time-api.time-api,cloud1_viriciti_com,NON_ARAALI_EGRESS,[443],AMAZON-02,443,32,52.218.92.232,2022-11-16T03:26:30Z,"id://cloud1_viriciti_com,:default.time-api.tim...",NaN
3,EXTERNAL_COMMUNICATION_ATTEMPTED,EXISTING_PROCESS_COMPROMISED,1,OPEN,default.delta-api.delta-api,/usr/local/bin/python3.8,default,sh,uvicorn,default.delta-api.delta-api,cloud1_viriciti_com,NON_ARAALI_EGRESS,[443],AMAZON-02,443,32,52.218.1.80,2022-11-16T03:26:46Z,"id://cloud1_viriciti_com,:default.delta-api.de...",NaN
4,EXTERNAL_COMMUNICATION_ATTEMPTED,EXISTING_PROCESS_COMPROMISED,1,OPEN,default.external-dns.external-dns,/bin/external-dns,default,containerd-shim,external-dns,default.external-dns.external-dns,cloud1_viriciti_com,NON_ARAALI_EGRESS,[443],AMAZON-02,443,32,52.94.235.198,2022-11-16T03:26:47Z,"id://cloud1_viriciti_com,:default.external-dns...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,EXTERNAL_COMMUNICATION_ATTEMPTED,EXISTING_PROCESS_COMPROMISED,1,OPEN,nexus,/usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java,nexus,systemd,java,nexus,cp-qa-ore-nos,NON_ARAALI_EGRESS,[443],NaN,443,NaN,NaN,2022-12-07T08:53:00Z,"id://cp-qa-ore-nos,:nexus:,com.install4j.runti...",:sonatype-download.global.ssl.fastly.net:
280,EXTERNAL_COMMUNICATION_ATTEMPTED,EXISTING_PROCESS_COMPROMISED,1,OPEN,k8s-nodes,/usr/bin/containerd,k8s-nodes,systemd,containerd,k8s-nodes,dev_viriciti,NON_ARAALI_EGRESS,[443],NaN,443,NaN,NaN,2022-12-07T13:38:00Z,"id://dev_viriciti,:k8s-nodes:,containerd,syste...",:registry-1.docker.io:
281,EXTERNAL_COMMUNICATION_ATTEMPTED,EXISTING_PROCESS_COMPROMISED,1,OPEN,k8s-nodes,/usr/bin/containerd,k8s-nodes,systemd,containerd,k8s-nodes,dev_viriciti,NON_ARAALI_EGRESS,[443],NaN,443,NaN,NaN,2022-12-07T15:16:00Z,"id://dev_viriciti,:k8s-nodes:,containerd,syste...",:ghcr.io:
282,EXTERNAL_COMMUNICATION_ATTEMPTED,EXISTING_PROCESS_COMPROMISED,1,OPEN,k8s-nodes,/usr/bin/containerd,k8s-nodes,systemd,containerd,k8s-nodes,dev_viriciti,NON_ARAALI_EGRESS,[443],NaN,443,NaN,NaN,2022-12-07T15:16:00Z,"id://dev_viriciti,:k8s-nodes:,containerd,syste...",:pkg-containers.githubusercontent.com:


In [126]:
list(alerts_to_csv(alerts))[:10]

['alert_info.communication_alert_type=EXTERNAL_COMMUNICATION_ATTEMPTED, alert_info.process_alert_type=EXISTING_PROCESS_COMPROMISED, alert_info.re_open_count=1, alert_info.status=OPEN, client.araali.app=default.canveyor.canveyor, client.araali.binary_name=/usr/local/bin/node, client.araali.namespace=default, client.araali.parent_process=containerd-shim, client.araali.process=node, client.araali.unmapped_app=default.canveyor.canveyor, client.araali.zone=cloud1_viriciti_com, direction=NON_ARAALI_EGRESS, ports.active_ports=[443], server.non_araali.organization=AMAZON-02, server.non_araali.server.dst_port=443, server.non_araali.server.subnet.net_mask=32, server.non_araali.server.subnet.subnet=52.218.112.152, timestamp=2022-11-16T03:25:41Z, unique_id=id://cloud1_viriciti_com,:default.canveyor.canveyor:,node,containerd-shim,/usr/local/bin/node+++AMAZON-02:443+++false+++false',
 'alert_info.communication_alert_type=EXTERNAL_COMMUNICATION_ATTEMPTED, alert_info.process_alert_type=NEW_PROCESS_SEE

In [97]:
def alert_to_df(alerts):
    for a in alerts:
        yield {"obj": a, "direction": a["direction"], 
               "server_araali": "araali" in a["server"],
               "client_zone": a["client"]["araali"]["zone"],
               "server": a["server"]["non_araali"],
               "org": a["server"]["non_araali"].get("organization", "None"),
               "dns": a["server"]["non_araali"]["server"].get("dns_pattern", "None"),
              }

In [98]:
df = pd.DataFrame(alert_to_df(alerts))

In [99]:
df

,obj,direction,server_araali,client_zone,server,org,dns
0,{'alert_info': {'communication_alert_type': 'E...,NON_ARAALI_EGRESS,False,cloud1_viriciti_com,"{'organization': 'AMAZON-02', 'server': {'dst_...",AMAZON-02,None
1,{'alert_info': {'communication_alert_type': 'E...,NON_ARAALI_EGRESS,False,cloud1_viriciti_com,"{'organization': 'AMAZON-02', 'server': {'dst_...",AMAZON-02,None
2,{'alert_info': {'communication_alert_type': 'E...,NON_ARAALI_EGRESS,False,cloud1_viriciti_com,"{'organization': 'AMAZON-02', 'server': {'dst_...",AMAZON-02,None
3,{'alert_info': {'communication_alert_type': 'E...,NON_ARAALI_EGRESS,False,cloud1_viriciti_com,"{'organization': 'AMAZON-02', 'server': {'dst_...",AMAZON-02,None
4,{'alert_info': {'communication_alert_type': 'E...,NON_ARAALI_EGRESS,False,cloud1_viriciti_com,"{'organization': 'AMAZON-02', 'server': {'dst_...",AMAZON-02,None
...,...,...,...,...,...,...,...
279,{'alert_info': {'communication_alert_type': 'E...,NON_ARAALI_EGRESS,False,cp-qa-ore-nos,{'server': {'dns_pattern': ':sonatype-download...,None,:sonatype-download.global.ssl.fastly.net:
280,{'alert_info': {'communication_alert_type': 'E...,NON_ARAALI_EGRESS,False,dev_viriciti,{'server': {'dns_pattern': ':registry-1.docker...,None,:registry-1.docker.io:
281,{'alert_info': {'communication_alert_type': 'E...,NON_ARAALI_EGRESS,False,dev_viriciti,"{'server': {'dns_pattern': ':ghcr.io:', 'dst_p...",None,:ghcr.io:
282,{'alert_info': {'communication_alert_type': 'E...,NON_ARAALI_EGRESS,False,dev_viriciti,{'server': {'dns_pattern': ':pkg-containers.gi...,None,:pkg-containers.githubusercontent.com:


In [68]:
df2 = df[df.dns == "None"]

In [69]:
df2

,obj,direction,server_araali,server,org,dns
0,{'alert_info': {'communication_alert_type': 'E...,NON_ARAALI_EGRESS,False,"{'organization': 'AMAZON-02', 'server': {'dst_...",AMAZON-02,None
1,{'alert_info': {'communication_alert_type': 'E...,NON_ARAALI_EGRESS,False,"{'organization': 'AMAZON-02', 'server': {'dst_...",AMAZON-02,None
2,{'alert_info': {'communication_alert_type': 'E...,NON_ARAALI_EGRESS,False,"{'organization': 'AMAZON-02', 'server': {'dst_...",AMAZON-02,None
3,{'alert_info': {'communication_alert_type': 'E...,NON_ARAALI_EGRESS,False,"{'organization': 'AMAZON-02', 'server': {'dst_...",AMAZON-02,None
4,{'alert_info': {'communication_alert_type': 'E...,NON_ARAALI_EGRESS,False,"{'organization': 'AMAZON-02', 'server': {'dst_...",AMAZON-02,None
...,...,...,...,...,...,...
261,{'alert_info': {'communication_alert_type': 'E...,NON_ARAALI_EGRESS,False,"{'organization': 'ISC-AS', 'server': {'dst_por...",ISC-AS,None
262,{'alert_info': {'communication_alert_type': 'E...,NON_ARAALI_EGRESS,False,"{'organization': 'MAX-GIGAPOP', 'server': {'ds...",MAX-GIGAPOP,None
263,{'alert_info': {'communication_alert_type': 'E...,NON_ARAALI_EGRESS,False,"{'organization': 'VGRS-AC20', 'server': {'dst_...",VGRS-AC20,None
264,{'alert_info': {'communication_alert_type': 'E...,NON_ARAALI_EGRESS,False,"{'organization': 'WIDE Project', 'server': {'d...",WIDE Project,None


In [94]:
pd.Series([a["client"]["araali"]["zone"] for a in df[(df.org == "AMAZON-02") & (df.dns == "None")].obj]
            ).value_counts()

cloud1_viriciti_com                  77
cp-qa-ore-nos                        18
dev_viriciti                         14
dev_viriciti_com                      3
eks-qa-ore-supplierportal-cluster     3
eu-central-1                          1
dtype: int64

In [80]:
print(yaml.dump([a["server"]["non_araali"]["server"]["subnet"]["subnet"] 
                 for a in df[(df.org == "None") & (df.dns == "None")].obj]))

- 104.16.22.35
- 104.16.26.35
- 104.17.25.14
- 104.18.70.113
- 104.18.72.113
- 104.17.24.14
- 104.18.70.113
- 104.18.72.113
- 52.218.118.42
- 104.16.24.35
- 104.16.25.35
- 52.218.92.120



# Frequency Counts

In [70]:
df2.org.value_counts()

AMAZON-02                                                      116
None                                                            12
GOOGLE                                                           7
AMAZON-AES                                                       5
FASTLY                                                           3
CLOUDFLARENET                                                    3
GITHUB                                                           3
DNIC-AS-01508                                                    2
CLT-NIC                                                          2
ISC-AS                                                           2
BROOT-AS                                                         2
NARC-EROOT                                                       2
MAX-GIGAPOP                                                      2
COGENT-2149                                                      2
IMRS                                                          

In [66]:
df.dns.value_counts()

None                                                         178
:openexchangerates.org:                                        3
:packagist.org:                                                3
:recaptchaenterprise.googleapis.com:                           3
:api-3t.sandbox.paypal.com:                                    3
                                                            ... 
:charts.rook.io:                                               1
:c5984d626704dc40b3470d73f00acd8a.us-west-2.aws.found.io:      1
:api.service.hmrc.gov.uk:                                      1
:ghcr.io:                                                      1
:csdb-01.cloud1.viriciti.com:                                  1
Name: dns, Length: 89, dtype: int64

In [60]:
df.org.value_counts()

None                                                           118
AMAZON-02                                                      116
GOOGLE                                                           7
AMAZON-AES                                                       5
FASTLY                                                           3
CLOUDFLARENET                                                    3
GITHUB                                                           3
DNIC-AS-01508                                                    2
CLT-NIC                                                          2
ISC-AS                                                           2
BROOT-AS                                                         2
NARC-EROOT                                                       2
MAX-GIGAPOP                                                      2
COGENT-2149                                                      2
IMRS                                                          

In [100]:
df.client_zone.value_counts()

cloud1_viriciti_com                  124
cp-qa-ore-nos                         43
cp-qa-fra-nos                         26
dev_viriciti                          24
eu-central-1                          19
cp-qa-mon-nos                         17
viriciti                               9
cp-qa-ore-pci                          6
dev_viriciti_com                       4
cp-qa-mon-pci                          4
eks-qa-ore-supplierportal-cluster      3
cp-qa-fra-pci                          3
cp-it-dev-k8s-cluster                  2
Name: client_zone, dtype: int64

In [38]:
df.server_araali.value_counts()

False    284
Name: server_araali, dtype: int64

In [39]:
df.direction.value_counts()

NON_ARAALI_EGRESS    284
Name: direction, dtype: int64